In [1]:
## Instruction: 
## https://darksky.net/dev/docs
## https://darksky.net/dev/docs/libraries

In [3]:
import json
import requests
import numpy as np
import pandas as pd
import time, calendar
from datetime import datetime, timedelta

In [21]:
# generate the list of days defined between start and end day
def calendar_test(day, count, date_list = None):
    if date_list is None:
        date_list = []
    if count == 0:
        return date_list
    count -= 1
    day = day + timedelta(days=1)
    date = day.isoformat().split('T')
    date_list.append(" ".join(date))
    
    calendar_test(day,count,date_list)
    
    return date_list

# convert utc time to epoch time
def convert_utc_to_epoch(timestamp_string):
    '''Use this function to convert utc to epoch'''
    timestamp = datetime.strptime(timestamp_string, '%Y-%m-%d %H:%M:%S')
    epoch = int(calendar.timegm(timestamp.utctimetuple()))

    return epoch

# convert epoch time to YYYYMMDDHHMM format (year-month-day-hour-minute)
def date_time_convert(item):
    
    date_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(item)).split(' ')
    date = date_time[0].split('-')
    time_ = date_time[1].split(':')
    date = 10000*int(date[0])+100*int(date[1])+1*int(date[2])
    time_ = 100*int(time_[0])+1*int(time_[1])
    date_time = date*10000+time_
    
    return date_time


def darksky_api_call(start_day,end_day):
    start_day = datetime.strptime(start_day, '%Y-%m-%d')
    end_day = datetime.strptime(end_day, '%Y-%m-%d')
    total_day = abs((end_day - start_day).days)

    day_list = calendar_test(start_day,total_day)

    # darksky website
    darksky = "https://api.darksky.net/forecast/"
    # API key
    key = "69fcefff98d6c6a0f9b0ebfec1e6f2cc"
    # Position: Williamstown, MA    
    lon_lat = "/42.719949, -73.207822,"
    # Offset
    offset = "?exclude=minutely,flags"

    # define parameter you want to collect
    info_list = ['time','summary','icon','precipIntensity','precipProbability','temperature','apparentTemperature',
                 'dewPoint','humidity','pressure','windSpeed','windGust','windBearing','cloudCover','uvIndex',
                 'visibility']
    info = {para:[] for para in info_list}

    # API calling
    for day in day_list:
        print(day)
    
        epoch_time = convert_utc_to_epoch(day)    
        link = darksky + key + lon_lat + str(epoch_time) + offset

        res = requests.get(link)
        data = res.json()
    
        if 'hourly' not in data:
            continue
        elif 'data' not in data['hourly']:
            continue
    
        for item in data['hourly']['data']:
            epoch = item['time']
            date_time = date_time_convert(item['time'])
            # parameters collected
            for para in info_list:
                if para not in item:
                    info[para].append(float('nan'))
                else:
                    info[para].append(item[para])
        
    file = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in info.items() ]))
    
    return file

In [24]:
start_day = '2017-9-12'
end_day = '2020-2-12'
start_day = datetime.strptime(start_day, '%Y-%m-%d')
end_day = datetime.strptime(end_day, '%Y-%m-%d')
total_day = abs((end_day - start_day).days)

day_list = calendar_test(start_day,total_day)
day_list

['2017-09-13 00:00:00',
 '2017-09-14 00:00:00',
 '2017-09-15 00:00:00',
 '2017-09-16 00:00:00',
 '2017-09-17 00:00:00',
 '2017-09-18 00:00:00',
 '2017-09-19 00:00:00',
 '2017-09-20 00:00:00',
 '2017-09-21 00:00:00',
 '2017-09-22 00:00:00',
 '2017-09-23 00:00:00',
 '2017-09-24 00:00:00',
 '2017-09-25 00:00:00',
 '2017-09-26 00:00:00',
 '2017-09-27 00:00:00',
 '2017-09-28 00:00:00',
 '2017-09-29 00:00:00',
 '2017-09-30 00:00:00',
 '2017-10-01 00:00:00',
 '2017-10-02 00:00:00',
 '2017-10-03 00:00:00',
 '2017-10-04 00:00:00',
 '2017-10-05 00:00:00',
 '2017-10-06 00:00:00',
 '2017-10-07 00:00:00',
 '2017-10-08 00:00:00',
 '2017-10-09 00:00:00',
 '2017-10-10 00:00:00',
 '2017-10-11 00:00:00',
 '2017-10-12 00:00:00',
 '2017-10-13 00:00:00',
 '2017-10-14 00:00:00',
 '2017-10-15 00:00:00',
 '2017-10-16 00:00:00',
 '2017-10-17 00:00:00',
 '2017-10-18 00:00:00',
 '2017-10-19 00:00:00',
 '2017-10-20 00:00:00',
 '2017-10-21 00:00:00',
 '2017-10-22 00:00:00',
 '2017-10-23 00:00:00',
 '2017-10-24 00:

In [22]:
# define start and end day
start_day = '2017-9-12'
end_day = '2020-2-12'

file = darksky_api_call(start_day,end_day)

# comment out if you want to save the data
# export_csv = file.to_csv ("file_name.csv", index = True, header=True)

2017-09-13 00:00:00
2017-09-14 00:00:00
2017-09-15 00:00:00
2017-09-16 00:00:00
2017-09-17 00:00:00
2017-09-18 00:00:00
2017-09-19 00:00:00
2017-09-20 00:00:00
2017-09-21 00:00:00
2017-09-22 00:00:00
2017-09-23 00:00:00
2017-09-24 00:00:00
2017-09-25 00:00:00
2017-09-26 00:00:00
2017-09-27 00:00:00
2017-09-28 00:00:00
2017-09-29 00:00:00
2017-09-30 00:00:00
2017-10-01 00:00:00
2017-10-02 00:00:00
2017-10-03 00:00:00
2017-10-04 00:00:00
2017-10-05 00:00:00
2017-10-06 00:00:00
2017-10-07 00:00:00
2017-10-08 00:00:00
2017-10-09 00:00:00
2017-10-10 00:00:00
2017-10-11 00:00:00
2017-10-12 00:00:00


KeyboardInterrupt: 

In [9]:
# run this code if you want to convert epoch time to YYYYMMDDHHMM format (year-month-day-hour-minute)
date = []
for i in file['time']:
    date.append(date_time_convert(i))
    
file['date_time'] = date

# comment out if you want to save the data
# export_csv = file.to_csv ("file_name.csv", index = True, header=True)

In [12]:
export_csv = file.to_csv ("weather_data.csv", index = False, header=True)